In [1]:
import numpy as np
from scipy.linalg import solve

In [2]:
def active_set_algorithm(G, c, A, b, x0, tol=1e-6, max_iter=100):

    x = x0
    m, n = A.shape
    active_set = []

    for k in range(max_iter):
        # Check for violated constraints
        residuals = A @ x - b
        violated = np.where(residuals > tol)[0]
        # Compute gradient
        grad = G @ x + c

        if len(violated) == 0:
            # If there are no violated constraints, check optimality conditions
            if len(active_set) == 0:
                p = -solve(G, grad)
            else:
                A_eq = A[active_set]
                try:
                    W = np.block([[G, A_eq.T], [A_eq, np.zeros((len(active_set), len(active_set)))]])
                    rhs = np.block([-grad, np.zeros(len(active_set))])
                    sol = solve(W, rhs)
                    p = sol[:n]
                    lag_mul = sol[n:]
                except np.linalg.LinAlgError:
                    p = np.zeros_like(x)

            # Check for optimality (Lagrange multipliers)
                if np.all(lag_mul >= -tol):
                    # print(f'Converged in {k} iterations.')
                    return x
        else:
            p = np.zeros_like(x)

        if np.linalg.norm(p) < tol:
            # Check if Lagrange multipliers are non-negative
            if len(active_set) == 0 or np.all(lag_mul >= -tol):
                # print(f'Converged in {k} iterations.')
                return x
            min_lag_mul_idx = np.argmin(lag_mul)
            active_set.pop(min_lag_mul_idx)
        else:
            # Take a step along p
            alpha = 1.0
            active_set_candidate = None
            for j in range(m):
                if j not in active_set:
                    Aj_p = A[j] @ p
                    if Aj_p < 0:
                        # Calculate the step size alpha that would violate this constraint
                        alpha_j = -(A[j] @ x - b[j]) / Aj_p
                        if alpha_j < alpha:
                            alpha = alpha_j
                            active_set_candidate = j
            # Update solution
            x = x + alpha * p

            if active_set_candidate is not None:
                active_set.append(active_set_candidate)
                
    raise ValueError('Active set algorithm did not converge.')

In [3]:
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([[1, -2], [-1, -2], [-1, 2], [1, 0], [0, 1]])
b = np.array([-2, -6, -2, 0, 0])
x0 = np.array([2, 0])

In [ ]:
optimal_x = active_set_algorithm(G, c, A, b, x0)

In [22]:
print("Optimal solution:", optimal_x)

Optimal solution: [1.4, 1.7]


In [4]:
tol=1e-6
residuals = A @ x0 - b
violated = np.where(residuals < tol)[0]

In [5]:
print(residuals)
print(violated)

[4 4 0 2 0]
[2 4]


In [38]:
active_set = violated

In [26]:
active_set = [4]

In [7]:
m, n = A.shape

In [40]:
A_eq = np.array(A[4])

In [33]:
A_eq = np.array([A[2], A[4]])

In [34]:
A_eq

array([[-1,  2],
       [ 0,  1]])

In [35]:
A_eq.T

array([[-1,  0],
       [ 2,  1]])

In [39]:
W = np.block([[G, A_eq.T], [A_eq, np.zeros((len(active_set), len(active_set)))]])

In [11]:
W

array([[ 2.,  0., -1.,  0.],
       [ 0.,  2.,  2.,  1.],
       [-1.,  2.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

In [12]:
grad = G @ x0 + c

In [13]:
grad

array([ 2, -5])

In [14]:
rhs = np.block([-grad, np.zeros(len(active_set))])

In [15]:
rhs

array([-2.,  5.,  0.,  0.])

In [16]:
sol = -solve(W, rhs)

In [17]:
sol

array([-2.22044605e-16, -0.00000000e+00, -2.00000000e+00, -1.00000000e+00])

In [23]:
p = sol[:n]
lag_mul = sol[n:]

In [24]:
lag_mul

array([-2., -1.])

In [25]:
p

array([-2.22044605e-16, -0.00000000e+00])

In [20]:
min_lag_mul_idx = np.argmin(lag_mul)
active_set = np.delete(active_set, min_lag_mul_idx)

In [41]:
x = x0 + p

In [42]:
x

array([2., 0.])

In [50]:
alpha = 1.0
active_set_candidate = None

In [51]:
active_set

array([4])

In [61]:
# if j not in active_set:
Aj_p = A[3] @ p
if Aj_p < 0:
    alpha_j = -(A[3] @ x0 - b[3]) / Aj_p
    if alpha_j < alpha:
        alpha = alpha_j
        active_set_candidate = j

In [62]:
print(active_set_candidate)

None


In [63]:
x0 = x0 + alpha * p

In [74]:
x0

array([2., 0.])

In [7]:
def active_set_qp(G, c, A_eq=None, b_eq=None, A_ineq=None, b_ineq=None, x0=None, tol=1e-8, max_iter=100):
    """
    Active-Set Method for solving convex QP:
    min (1/2)x^T G x + c^T x
    subject to A_eq x = b_eq (equality constraints)
               A_ineq x <= b_ineq (inequality constraints)
    Arguments:
    G: Quadratic coefficient matrix (must be positive semidefinite).
    c: Linear coefficient vector.
    A_eq: Equality constraint matrix.
    b_eq: Equality constraint RHS vector.
    A_ineq: Inequality constraint matrix.
    b_ineq: Inequality constraint RHS vector.
    x0: Initial feasible solution (optional).
    tol: Tolerance for convergence.
    max_iter: Maximum number of iterations.
    """
    # Initialize variables
    n = G.shape[0]  # Number of variables
    x = x0 if x0 is not None else np.zeros(n)
    
    # Working sets for constraints
    if A_eq is not None:
        W_eq = np.arange(A_eq.shape[0])  # Equality constraints are always active
    else:
        W_eq = np.array([])

    W = W_eq.tolist()  # Start with equality constraints in the working set
    
    # Preprocessing for inequality constraints
    if A_ineq is not None:
        # Check feasibility of the initial point
        if np.any(A_ineq @ x - b_ineq > tol):
            raise ValueError("Initial point is not feasible for inequality constraints.")
    
    for k in range(max_iter):
        # Step 1: Solve the QP subproblem for the current working set
        A_w = np.vstack([A_eq, A_ineq[W]]) if W else A_eq
        b_w = np.hstack([b_eq, b_ineq[W]]) if W else b_eq
        
        KKT_matrix = np.block([
            [G, A_w.T],
            [A_w, np.zeros((len(W), len(W)))]
        ])
        KKT_rhs = np.hstack([-c, b_w])
        
        try:
            solution = solve(KKT_matrix, KKT_rhs)
        except np.linalg.LinAlgError:
            raise ValueError("KKT system is singular.")
        
        p = solution[:n]  # Step direction
        lambda_w = solution[n:]  # Lagrange multipliers for active set
        
        # Check for optimality
        if np.linalg.norm(p) < tol:
            # Check KKT conditions
            if all(lam >= -tol for lam in lambda_w):  # Feasibility and nonnegative multipliers
                return x  # Optimal solution found
            else:
                # Remove the most negative multiplier from the active set
                most_negative = np.argmin(lambda_w)
                W.pop(most_negative)
        else:
            # Step 2: Compute step length α_k
            alpha = 1.0  # Full step by default
            blocking_constraints = []
            if A_ineq is not None:
                for i in range(A_ineq.shape[0]):
                    if i not in W and A_ineq[i] @ p < -tol:  # Check blocking condition
                        alpha_i = (b_ineq[i] - A_ineq[i] @ x) / (A_ineq[i] @ p)
                        if alpha_i < alpha:
                            alpha = alpha_i
                            blocking_constraints = [i]
            
            # Step 3: Update x and working set
            x = x + alpha * p
            if alpha < 1.0:  # Add blocking constraint to working set
                W.extend(blocking_constraints)
    
    raise ValueError("Maximum number of iterations reached without convergence.")


In [6]:
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A_eq = np.array([[1, 1]])
b_eq = np.array([1])
A_ineq = np.array([[1, 0], [0, 1]])
b_ineq = np.array([0, 0])
x0 = np.array([0.5, 0.5])

In [19]:
def active_set_qp_debug(G, c, A_eq=None, b_eq=None, A_ineq=None, b_ineq=None, x0=None, tol=1e-8, max_iter=100):
    """
    Debugged version of the Active-Set Method for solving convex QP:
    """
    n = G.shape[0]  # Number of variables
    x = x0 if x0 is not None else np.zeros(n)
    
    # Active sets for constraints
    if A_eq is not None:
        W_eq = np.arange(A_eq.shape[0])  # Equality constraints are always active
    else:
        W_eq = np.array([])

    W = W_eq.tolist()  # Start with equality constraints in the working set
    
    if A_ineq is not None:
        if np.any(A_ineq @ x - b_ineq > tol):
            raise ValueError("Initial point is not feasible for inequality constraints.")
    
    for k in range(max_iter):
        # Create working set matrix
        A_w = np.vstack([A_eq, A_ineq[W]]) if W else A_eq
        b_w = np.hstack([b_eq, b_ineq[W]]) if W else b_eq
        
        if A_w is None:
            A_w = np.zeros((0, n))
            b_w = np.zeros(0)
        
        # Construct the KKT system
        try:
            KKT_matrix = np.block([
                [G, A_w.T],
                [A_w, np.zeros((A_w.shape[0], A_w.shape[0]))]
            ])
            KKT_rhs = np.hstack([-c, b_w])
            solution = solve(KKT_matrix, KKT_rhs, assume_a='sym')
        except np.linalg.LinAlgError:
            raise ValueError("KKT system is singular or poorly conditioned.")
        
        p = solution[:n]  # Step direction
        lambda_w = solution[n:]  # Lagrange multipliers
        
        # Check for optimality
        if np.linalg.norm(p) < tol:
            if all(lam >= -tol for lam in lambda_w):
                return x  # Optimal solution
            else:
                most_negative = np.argmin(lambda_w)
                W.pop(most_negative)
        else:
            # Compute step length
            alpha = 1.0
            blocking_constraints = []
            if A_ineq is not None:
                for i in range(A_ineq.shape[0]):
                    if i not in W and A_ineq[i] @ p < -tol:
                        alpha_i = (b_ineq[i] - A_ineq[i] @ x) / (A_ineq[i] @ p)
                        if alpha_i < alpha:
                            alpha = alpha_i
                            blocking_constraints = [i]
            
            x = x + alpha * p
            if alpha < 1.0:
                W.extend(blocking_constraints)
    
    raise ValueError("Maximum iterations reached without convergence.")

# Test the debugged implementation
solution_debugged = active_set_qp_debug(G, c, A_eq=None, b_eq=None, A_ineq=A, b_ineq=b, x0=x0)

solution_debugged


ValueError: Initial point is not feasible for inequality constraints.

In [21]:
optimal_x = [1.400, 1.700]